[Kth Largest Element in an Array](https://leetcode.com/problems/kth-largest-element-in-an-array/)


Given an integer array nums and an integer k, return the kth largest element in the array.

Note: It is the kth largest element in the sorted order, not the kth distinct element.
 

Example 1:
```
Input: nums = [3,2,1,5,6,4], k = 2
Output: 5
```

Example 2:
```
Input: nums = [3,2,3,1,2,4,5,5,6], k = 4
Output: 4
```

**Constraints**:

* 1 <= k <= nums.length <= 104
* $-10^4 <= nums[i] <= 10^4$

Inspired by https://leetcode.com/problems/kth-largest-element-in-an-array/discuss/1349609

## Approach 1: MinHeap

We use minHeap to keep up to k smallest elements of the nums array. Then top of the minHeap is the k largest element.

Complexity:  
* Time: O(NlogK)  
* Space: O(K)


In [5]:
from typing import List

class Solution:
    def findKthLargest(self, nums: List[int], k: int) -> int:
        minHeap = []
        for x in nums:
            heappush(minHeap, x)
            if len(minHeap) > k:
                heappop(minHeap)
        return minHeap[0]

In fact, python already has this method in the library
> heapq.nlargest(n, iterable, key=None)  
Return a list with the n largest elements from the dataset defined by iterable. key, if provided, specifies a function of one argument that is used to extract a comparison key from each element in iterable (for example, key=str.lower). Equivalent to: sorted(iterable, key=key, reverse=True)[:n].

```python
import heapq

class Solution:
    def findKthLargest(self, nums, k):
        return heapq.nlargest(k, nums)[-1]
```

## Approach 2: Quick Select*

Complexity:

* Time: O(N) in the avarage case, O(N^2) in the worst case. Worst case happens when:
k = len(nums) and pivot is always the smallest element, so it divides array by [zero elements in the small, 1 element in the equal, n-1 elements in the large], so it always goes to the right side with n-1 elements each time.
k = 1 and pivot is always the largest element, so it divides array by [n-1 elements in the small, 1 element in the equal, zero elements in the large], so it always goes to the left side with n-1 elements reach time.

* Space: O(N)

In [6]:
class Solution:
    def findKthLargest(self, nums: List[int], k: int) -> int:
        return self.findKthSmallest(nums, len(nums) - k + 1)

    def findKthSmallest(self, nums, k):
        if len(nums) <= 1: return nums[0]
        pivot = random.choice(nums)
        small = [x for x in nums if x < pivot]
        equal = [x for x in nums if x == pivot]
        large = [x for x in nums if x > pivot]
        if k <= len(small):
            return self.findKthSmallest(small, k)
        if k <= len(small) + len(equal):
            return pivot
        return self.findKthSmallest(large, k - len(small) - len(equal))

## Approach 3: Quick Select (Recusion & Use In-Space memory)

Complexity:

* Time: O(N) in the avarage case, O(N^2) in the worst case.
* Space: O(N), recursion depth can up to O(N).

In [7]:
class Solution:
    def findKthLargest(self, nums: List[int], k: int) -> int:
        return self.findKthSmallest(nums, 0, len(nums) - 1, len(nums) - k + 1 - 1)

    def findKthSmallest(self, nums, left, right, k):  # k is one-base indexing
        def partition(left, right, pivotIndex):
            pivot = nums[pivotIndex]
            
            # Move pivot to the right most
            nums[right], nums[pivotIndex] = nums[pivotIndex], nums[right]
            pivotIndex = left
            
            # Swap elements less than pivot to the left
            for i in range(left, right):
                if nums[i] < pivot:
                    nums[pivotIndex], nums[i] = nums[i], nums[pivotIndex]
                    pivotIndex += 1
                    
            # Move pivot to the right place
            nums[pivotIndex], nums[right] = nums[right], nums[pivotIndex]
            return pivotIndex
        
        if left == right:
            return nums[left]
        
        pivotIndex = random.randint(left, right)  # Rand between [left, right]
        pivotIndex = partition(left, right, pivotIndex)
        if pivotIndex == k:
            return nums[pivotIndex]
        if k < pivotIndex:
            return self.findKthSmallest(nums, left, pivotIndex - 1, k)
        return self.findKthSmallest(nums, pivotIndex + 1, right, k)

## Approach 4: Quick select (In place)

Finally the overall algorithm is quite straightforward :

* Choose a random pivot.
* Use a partition algorithm to place the pivot into its perfect position pos in the sorted array, move smaller elements to the left of pivot, and larger or equal ones - to the right.
* Compare pos and N - k to choose the side of array to proceed recursively.

In [8]:
class Solution:
    def findKthLargest(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        def partition(left, right, pivot_index):
            pivot = nums[pivot_index]
            # 1. move pivot to end
            nums[pivot_index], nums[right] = nums[right], nums[pivot_index]  
            
            # 2. move all smaller elements to the left
            store_index = left
            for i in range(left, right):
                if nums[i] < pivot:
                    nums[store_index], nums[i] = nums[i], nums[store_index]
                    store_index += 1

            # 3. move pivot to its final place
            nums[right], nums[store_index] = nums[store_index], nums[right]  
            
            return store_index
        
        def select(left, right, k_smallest):
            """
            Returns the k-th smallest element of list within left..right
            """
            if left == right:       # If the list contains only one element,
                return nums[left]   # return that element
            
            # select a random pivot_index between 
            pivot_index = random.randint(left, right)     
                            
            # find the pivot position in a sorted list   
            pivot_index = partition(left, right, pivot_index)
            
            # the pivot is in its final sorted position
            if k_smallest == pivot_index:
                 return nums[k_smallest]
            # go left
            elif k_smallest < pivot_index:
                return select(left, pivot_index - 1, k_smallest)
            # go right
            else:
                return select(pivot_index + 1, right, k_smallest)

        # kth largest is (n - k)th smallest 
        return select(0, len(nums) - 1, len(nums) - k)